# Exercise: Titanic Bayes

###Using the Titanic dataset, clean up the data (handle missing values either by removal or filling, and transforming non-numerical data into number values) and then build Gaussian and Bernoulli Naive Bayes models to predict Titanic passengers' survival status (1=survived, 0=did not survive). Compare the two models against each other. Did one model perform better than the other? How does the performance of these two models compare to the other classification algorithms, logistic regression and decision trees?

For a bonus challenge, try different methods of preparing your data (cleaning, choosing rows/columns) to see if that affects your results.

*To see an example of predictive output of the logistic regression and decision trees, run the code in the notebooks for the Lv 1 Module 8: Logistic Regression and Module 9: Decision Trees notebooks (Links to an external site.)Links to an external site..

Upload your Jupyter notebook to Github and submit the URL to turn in this assignment.



# TODO
### 1. clean up the data (handle missing values either by removal or filling
###2.  transforming non-numerical data into number values)
###3.then build Gaussian and Bernoulli Naive Bayes models to predict Titanic passengers' survival status (1=survived, 0=did not survive). ###4. Compare the two models against each other

In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data 

filename = "titanic.xls"
df = pd.read_excel(filename) 


df.head() #first 5 rows

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.2+ KB


In [4]:
np.logical_not(df.isnull()).sum()

pclass       1309
survived     1309
name         1309
sex          1309
age          1046
sibsp        1309
parch        1309
ticket       1309
fare         1308
cabin         295
embarked     1307
boat          486
body          121
home.dest     745
dtype: int64

In [7]:
#find columns that have missing values
df.isnull().sum()

#np.logical_not(df.isnull()).sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [6]:
np.logical_not(df.isnull()).sum()

pclass       1309
survived     1309
name         1309
sex          1309
age          1046
sibsp        1309
parch        1309
ticket       1309
fare         1308
cabin         295
embarked     1307
boat          486
body          121
home.dest     745
dtype: int64

In [8]:
#rows where the age is missing
missing_age = df.loc[df['age'].isnull()]
missing_age.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
15,1,0,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.9250,NaN,S,NaN,NaN,"New York, NY"
37,1,1,"Bradley, Mr. George (""George Arthur Brayton"")",male,NaN,0,0,111427,26.5500,NaN,S,9,NaN,"Los Angeles, CA"
40,1,0,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.6000,NaN,C,NaN,NaN,"Philadelphia, PA"
46,1,0,"Cairns, Mr. Alexander",male,NaN,0,0,113798,31.0000,NaN,S,NaN,NaN,NaN
59,1,1,"Cassebeer, Mrs. Henry Arthur Jr (Eleanor Genev...",female,NaN,0,0,17770,27.7208,NaN,C,5,NaN,"New York, NY"


In [9]:
#get index numbers of missing rows - we'll use this later
mals = list(missing_age.index)

In [10]:
#table of avg age of passenger by survival status, sex, and passenger class
df.groupby(['survived', 'sex', 'pclass'])['age'].mean()

survived  sex     pclass
0         female  1         35.200000
                  2         34.090909
                  3         23.418750
          male    1         43.658163
                  2         33.092593
                  3         26.679598
1         female  1         37.109375
                  2         26.711051
                  3         20.814815
          male    1         36.168240
                  2         17.449274
                  3         22.436441
Name: age, dtype: float64

In [11]:
#fill missing values for age based on survival status, sex, and passenger class
df['age'].fillna(df.groupby(['survived', 'sex', 'pclass'])['age'].transform('mean'), inplace=True)

In [12]:
#verify filled missing values 
df.iloc[mals].head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
15,1,0,"Baumann, Mr. John D",male,43.658163,0,0,PC 17318,25.9250,NaN,S,NaN,NaN,"New York, NY"
37,1,1,"Bradley, Mr. George (""George Arthur Brayton"")",male,36.168240,0,0,111427,26.5500,NaN,S,9,NaN,"Los Angeles, CA"
40,1,0,"Brewe, Dr. Arthur Jackson",male,43.658163,0,0,112379,39.6000,NaN,C,NaN,NaN,"Philadelphia, PA"
46,1,0,"Cairns, Mr. Alexander",male,43.658163,0,0,113798,31.0000,NaN,S,NaN,NaN,NaN
59,1,1,"Cassebeer, Mrs. Henry Arthur Jr (Eleanor Genev...",female,37.109375,0,0,17770,27.7208,NaN,C,5,NaN,"New York, NY"


In [13]:
#verify there are no more missing age values
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age             0
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [14]:
#missing values for 'embarked'
embark = df.loc[df['embarked'].isnull()]
embark

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
168,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,6,NaN,NaN
284,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,6,NaN,"Cincinatti, OH"


In [15]:
#save index for missing values to verify later
embarkls = list(embark.index)

In [16]:
#only 2 missing values so we'll fill with most common embarkation point
df['embarked'].value_counts()

S    914
C    270
Q    123
Name: embarked, dtype: int64

In [17]:
#fill missing values
df['embarked'].fillna('S', inplace=True)

In [18]:
#check that they're filled
df.iloc[embarkls]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
168,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S,6,NaN,NaN
284,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,S,6,NaN,"Cincinatti, OH"


In [19]:
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age             0
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        0
boat          823
body         1188
home.dest     564
dtype: int64

In [20]:
modeldf = df.drop(['name','ticket','fare', 'cabin', 'boat', 'body', 'home.dest'], axis=1)

In [21]:
#columns left in our dataframe
modeldf.columns

Index(['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch', 'embarked'], dtype='object')

In [22]:
#change sex values to binary
#female=0, male=1
modeldf['sex'] = modeldf['sex'].map({'female':0, 'male':1})
modeldf.head()

,pclass,survived,sex,age,sibsp,parch,embarked
0,1,1,0,29.0000,0,0,S
1,1,1,1,0.9167,1,2,S
2,1,0,0,2.0000,1,2,S
3,1,0,1,30.0000,1,2,S
4,1,0,0,25.0000,1,2,S


In [ ]:
#descriptive statistics
df.describe()

In [ ]:
#total number of students
total = len(df)

In [ ]:
#rows of students that passed the test
df_pass = df[df['grade'] >= 60]

#number of students that passed
numpass = len(df_pass)

In [ ]:
#rows of students that studied 5 hours or less
df_less5hr = df[df['hours'] <= 5]

#number of students that studied 5 hours or less
num_less5hr = len(df_less5hr)

In [ ]:
#rows of students that studied 5 hours or less and passed
df_5less_pass = df_pass.loc[df['hours'] <= 5]

#number of students that studied 5 hours or less and passed
num_5less_pass = len(df_5less_pass)

In [ ]:
#probability of passing the test
#number of students that passed divided by total number of students
P_pass = numpass/total
P_pass

In [ ]:
#probability of studying 5 hours or less
#number of students that studied 5 hours or less divided by total number of students
P_less5hr = num_less5hr/total
P_less5hr

In [ ]:
#probability of studying 5 hours or less given that you passed
#number of students that studied 5 hours or less given they passed, divided by total students that passed
P_5hr_pass = num_5less_pass/numpass
P_5hr_pass

In [ ]:
#SOLUTION: probability of passing given that you studied 5 hours or less

#probability of passing times probability of studying 5 hours or less given that you passed
#divded by probability of studying 5 hours or less
P_pass_less5hr = (P_pass * P_5hr_pass)/(P_less5hr)
P_pass_less5hr

#### The probability of a passing the test, given that a student studied 5 hours or less is about 93.5%. So a student only has a 6.5% chance of failing. That's not too bad; maybe the test is fairly easy.

***

## Naïve Bayes using Scikit-Learn

Let's use the same dataset above and build a Naïve Bayes classification model to predict student grades.

### Gaussian Naïve Bayes

There are different types of Naive Bayes functions and in the examples below, we will use Gaussian Bayes to build the predictive model. Gaussian Bayes uses conditional probability on data that is normally distributed.

In [ ]:
from sklearn.naive_bayes import GaussianNB   #import Gaussian Bayes modeling function
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
#check to see if there are any missing values
df.count()

In [ ]:
df.dtypes

In [ ]:
#create a dataframe with columns to use in the model
modeldf = df[['gender', 'age', 'exercise', 'hours', 'grade']]
modeldf.head()

In [ ]:
#transform gender column to binary values (0,1)
modeldf['gender'] = modeldf['gender'].map({'female': 0, 'male': 1})
modeldf.head()

In [ ]:
#see which features are correlated to each other
modeldf.corr()

In [ ]:
#create a column to label if a student passed or failed a test
modeldf['passed'] = np.where(df['grade']>= 60, 1, 0)

#drop grade column
modeldf.drop('grade', axis=1, inplace=True)

In [ ]:
#dataframe with predicting features
X = modeldf.drop('passed', axis=1)

#column of predictive target values
y = modeldf['passed']

In [ ]:
#create training and test data
#will leave test size at default (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=109)

In [ ]:
#initialize Gaussian Bayes classifier
gnb = GaussianNB()

In [ ]:
#train the model to learn trends
gnb.fit(X_train, y_train)

In [ ]:
#predictive score of the model on the training data
gnb.score(X_train, y_train)

In [ ]:
#test the model on unseen data
#score predictive values in variable
y_pred = gnb.predict(X_test)

In [ ]:
#Confusion matrix shows which values model predicted correctly vs incorrectly

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Failed', 'Predicted Passed'],
    index=['True Failed', 'True Passed']
)

cm

In [ ]:
#frequency of passed students to failed students in the test dataset
y_test.value_counts()

In [ ]:
#predictive score of the model on the test data
gnb.score(X_test, y_test)

In [ ]:
#predictive score of the model for each predictive category
print(classification_report(y_test, y_pred))

### Bernoulli's Naïve Bayes

Bernoull's Naïve Bayes classifier is best on a target variable that is binary (Boolean; True/False (1,0) values). Let's try this method on the dataset from the previous example.

In [ ]:
#import Bernoulli Naïve Bayes function from scikit-learn library
from sklearn.naive_bayes import BernoulliNB

In [ ]:
#initialize Bernoulli Naïve Bayes function to a variable
bnb = BernoulliNB()

In [ ]:
#build the model with training data
bnb.fit(X_train, y_train)

In [ ]:
#model's predictive score on the training data
bnb.score(X_train, y_train)

In [ ]:
#test the model on unseen data
#score predictive values in variable
y_pred = bnb.predict(X_test)

In [ ]:
#Confusion matrix shows which values model predicted correctly vs incorrectly

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Failed', 'Predicted Passed'],
    index=['True Failed', 'True Passed']
)

cm

In [ ]:
#predictive score of the model on the test data
bnb.score(X_test, y_test)

Overall, the model is really good at finding students that passed but in this test dataset, it didn't have enough data points to find the trend of predicting features for students that failed the test. One way to improve the results would be to decrease the size of the training data so that data points for failing students seem more significant. This dataset is also small, so new data with more students that failed could help the model see the trends for failing students. Lastly, it could just be that Naïve Bayes isn't the best model to use for the data and we should compare its results to other predictive classification models.